In [1]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader

In [2]:
dr = DataReader()
shot_df = dr.read_shot("shots-20201201010000", shot_number=1)
shot_df.head()

,timestamp,sequential_number,sequential_number_by_shot,displacement,load01,load02,load03,load04,shot_number,tags
0,1.606752e+09,0,0,49.171,0.568421,0.526316,0.597368,0.502632,1,"[異常A, 異常B]"
1,1.606752e+09,1,1,49.245,0.550000,0.526316,0.623684,0.507895,1,"[異常A, 異常B]"
2,1.606752e+09,2,2,49.136,0.568421,0.510526,0.660526,0.521053,1,"[異常A, 異常B]"
3,1.606752e+09,3,3,49.184,0.600000,0.502632,0.678947,0.521053,1,"[異常A, 異常B]"
4,1.606752e+09,4,4,49.149,0.578947,0.473684,0.671053,0.521053,1,"[異常A, 異常B]"


In [3]:
shots_meta_df = dr.read_shots_meta("shots-meta-20201201010000")
shots_meta_df.head()

,shot_number,spm,num_of_samples_in_cut_out
0,1,80.598572,3125.0
1,2,80.613753,3113.0
2,3,80.594235,3129.0
3,4,80.636509,3097.0
4,5,80.582338,3140.0


In [4]:
from analyze.hone_extract_features import *

spm = shots_meta_df[shots_meta_df.shot_number == 1].spm
print(f"spm: {spm}")

extract_features(shot_df, spm, max_load)

Loading BokehJS ...

spm: 0    80.598572
Name: spm, dtype: float64


([2182, 2181, 1975, 2019],
 [6.785456189524168, 5.385527501434686, 3.70495865877046, 4.012740422000842])

In [13]:
num_of_shots = len(shots_meta_df)
# print(f"num_of_shots: {num_of_shots}")

#for shot_number in range(1, num_of_shots+1):
for shot_number in range(1, 15): # small set
    shot_df = dr.read_shot("shots-20201201010000", shot_number=shot_number)
    spm = shots_meta_df[shots_meta_df.shot_number == shot_number].spm

    print(f"shot_number: {shot_number}, num_of_data: {len(shot_df)}, spm: {spm}")

    indies, values = extract_features(shot_df, spm, max_load)
    print(indies, values)

shot_number: 1, num_of_data: 3125, spm: 0    80.598572
Name: spm, dtype: float64
[2182, 2181, 1975, 2019] [6.785456189524168, 5.385527501434686, 3.70495865877046, 4.012740422000842]
shot_number: 2, num_of_data: 3113, spm: 1    80.613753
Name: spm, dtype: float64
[2053, 2050, 2205, 2204] [4.120647443029952, 3.5650173325164443, 4.264495176498813, 4.417101536658378]
shot_number: 3, num_of_data: 3129, spm: 2    80.594235
Name: spm, dtype: float64
[2017, 2036, 2175, 2174] [4.267050196346643, 3.7811680066507107, 5.823104689298365, 5.218945661365101]
shot_number: 4, num_of_data: 3097, spm: 3    80.636509
Name: spm, dtype: float64
[2128, 2126, 1924, 2022] [6.421476000312592, 5.360403888150232, 3.699402944629051, 4.262732642091777]
shot_number: 5, num_of_data: 3140, spm: 4    80.582338
Name: spm, dtype: float64
[2051, 2013, 2147, 2103] [4.2548288433741, 3.7862846030638098, 3.848599858150906, 4.321757864127532]
shot_number: 6, num_of_data: 3109, spm: 5    80.598572
Name: spm, dtype: float64
[219